In [1]:
import numpy as np
from itertools import count
from scipy.optimize import linprog
import time

def generate_constraint_matrix(num_scenarios,percentage): #generates the constraint matrix which takes into account the number of scenarios and the percentage of each scenario.
  constraint_matrices = [] #gets the value (from the function below) for the objective function and right hand side.
  constraints = [] #constrain value
  Slacks = [] #slack value
  variables_count = 3 + 6*num_scenarios #number variabes generated per scenario. 6 variables added per scenario while only X1,X2,X3 (the first 3 variables) stay the same.
  slacks_count = 1 + 4*num_scenarios #counts the amout of slacks that will be generated for every scenario. 1st contraint stays with the original scenario and only the next 4 constraints are repeated for every scenario
  c = [0] * (variables_count + slacks_count)
  b = [0] * (1 + 4*num_scenarios)

  for i in range(num_scenarios): #Generate constraints for each scenario
    for j in range(5):  # Number of constraints per scenario
      row = [0] * variables_count
      if i==0 and j == 0:
        row[0] = 1  # Coefficients for X1 in the first constraint
        row[1] = 1  # Coefficients for X2 in the first constraint
        row[2] = 1  # Coefficients for X3 in the first constraint
      elif i!=0 and j == 0:
        continue  # Skip the first constraint for subsequent matrices
      elif j == 1:
        row[0] = 2.5 * (1+percentage*((num_scenarios-1)/2)-percentage*i)   # Coefficients for X1 in the second constraint
        row[3+6*i] = 1  # Coefficients for X4 in the second constraint
        row[5+6*i] = -1  # Coefficients for X6 in the second constraint
      elif j == 2:
        row[1] = 3 * (1+percentage*((num_scenarios-1)/2)-percentage*i)  # Coefficients for X2 in the third constraint
        row[4+6*i] = 1  # Coefficients for X5 in the third constraint
        row[6+6*i] = -1  # Coefficients for X7 in the third constraint
      elif j == 3:
        row[2] = -20 * (1+percentage*((num_scenarios-1)/2)-percentage*i)  # Coefficients for X3 in the fourth constraint
        row[7+6*i] = 1  # Coefficients for X8 in the fourth constraint
        row[8+6*i] = 1  # Coefficients for X9 in the fourth constraint
      elif j == 4:
        row[7+6*i] = 1  # Coefficients for X8 in the fifth constraint

      constraints.append(row)

  for i in range(num_scenarios):
    # Generate constraints for each scenario
    for j in range(5):  # Number of constraints per scenario
      row = [0] * slacks_count
      if i==0 and j == 0:
        row[0] = 1  # Coefficients for s1 in the first constraint
      elif i!=0 and j == 0:
        continue  # Skip the first constraint for subsequent matrices
      elif j == 1:
        row[1+4*i] = -1  # Coefficients for s2 in the second constraint
      elif j == 2:
        row[2+4*i] = -1  # Coefficients for s3 in the third constraint
      elif j == 3:
        row[3+4*i] = 1  # Coefficients for s4 in the fourth constraint
      elif j == 4:
        row[4+4*i] = 1  # Coefficients for s5 in the fifth constraint

      Slacks.append(row)

  constraint_matrices = np.hstack((constraints,Slacks))# generates the matrix which consists of the objective function and Right hand side value

  c[0] = 150
  c[1] = 230
  c[2] = 260
  for i in range(num_scenarios):
    c[3 + (i*6)] = 238 * (1/num_scenarios)
    c[4 + (i*6)] = 210 * (1/num_scenarios)
    c[5 + (i*6)] = -170 * (1/num_scenarios)
    c[6 + (i*6)] = -150 * (1/num_scenarios)
    c[7 + (i*6)] = -36 * (1/num_scenarios)
    c[8 + (i*6)] = -10 * (1/num_scenarios)

  b[0] = 500
  for i in range(num_scenarios):
    b[1 + (i*4)] = 200
    b[2 + (i*4)] = 240
    b[3 + (i*4)] = 0
    b[4 + (i*4)] = 6000

  b = np.array(b).reshape(-1,1)

  return constraint_matrices, c, b

In [2]:
def generate_phase_one_lp_coefficients_from_constraints(A,b, c): #generates the matrix for the phase 1 of the simplex method.

  num_constraints, num_variables = A.shape
  x = np.ones((num_variables,1))
  artificial_matrix = np.eye(num_constraints)
  A_with_artificial = np.concatenate((A, artificial_matrix), axis=1)

  c_new = np.zeros(num_variables + num_constraints)
  c_new[:num_variables] = c
  c_new[num_variables:] = 1000000  # Coefficients for artificial variables

  coefficients = {
    'A': A_with_artificial,
    'b': b.reshape(-1,1),
    'c': c_new.reshape(-1,1)}

  return coefficients

In [12]:
num_scenarios = 89 #user can change the number of scenarios they would like to generate
percentage = 0.001 #user can chnage the percentage +/- for the number of scenarios generated.
A_pre,c_pre,b_pre = generate_constraint_matrix(num_scenarios,percentage)

In [13]:
beginning = time.time() #starts generating the time for the duration of the simplex method.

phase_one_coefficients = generate_phase_one_lp_coefficients_from_constraints(A_pre,b_pre,c_pre)

A = phase_one_coefficients['A']
b = phase_one_coefficients['b']
c = phase_one_coefficients['c']

normalX = A_pre.shape[1]
variableX = A.shape[1]
xB_Index = np.arange(normalX, variableX).reshape(1, variableX-normalX)
xN_Index = np.arange(0, normalX).reshape(1, A_pre.shape[1])

zero_X = np.zeros((A_pre.shape[1],1))
X = np.vstack((zero_X,b))

for i_count in count():
  B = A[:, xB_Index].reshape(A.shape[0],A.shape[0])
  cB = c[xB_Index, :].reshape(A.shape[0],1)
  w = np.linalg.solve(B.T, cB)

  reduced_cost_values = np.array([]) #generated the reduced cost values
  for i_N in xN_Index:
    cN_r = c[i_N,0]
    N_r = A[:, i_N]
    reduced_cost = cN_r - (w.T)@N_r
    reduced_cost_values = np.append(reduced_cost_values, reduced_cost)

  positive_r = 0 #if statement which exists the function if the reduced cost is greater than or = to 0. Otherwise continue running the code.
  for i in reduced_cost_values:
    if i >= 0:
      positive_r = positive_r + 1
  if positive_r == reduced_cost_values.shape[0]:
    Optimal_X = X
    Optimal_value = c.T @ X
    break

  for i_r, r in enumerate(reduced_cost_values):
    if r < 0:
      enter_variable = xN_Index[0,i_r]
      break
  xB_direction = np.linalg.solve(B, -A[:, enter_variable].reshape(A.shape[0],1))# This is the direction vector in which the new variable enters
  e = np.array([])
  for num in range(xN_Index.shape[1]):
    if xN_Index[0,num] == enter_variable:
      e = np.append(e,1)
    else:
      e = np.append(e,0)
  direction = np.vstack((xB_direction, e.reshape(xN_Index.shape[1],1)))
  total_index = np.hstack((xB_Index,xN_Index))
  total_index_flat = total_index.flatten()
  sorting_index = np.argsort(total_index_flat)
  direction_sorted = direction[sorting_index]

  a = np.array([])
  a_index = np.array([])
  for i_d,d_value in enumerate(direction):
    if d_value < 0:
      a = np.append(a, -1*X[xB_Index[0,i_d],0]/d_value)
    else:
      a = np.append(a, 100000) #A reference alpha of 100,000 was used to compare the alpha values

  a_min = np.min(a)
  exit_variable = total_index[0,np.where(a == a_min)[0][0]]
  X_new = X + (a_min * direction_sorted)
  X = X_new
  xB_Index = np.append(xB_Index, enter_variable)
  mask = (xB_Index != exit_variable)
  xB_Index = xB_Index[mask]
  xB_Index = np.sort(xB_Index).reshape(1,xB_Index.shape[0])

  xN_Index = np.append(xN_Index, exit_variable)
  mask = (xN_Index != enter_variable)
  xN_Index = xN_Index[mask]
  xN_Index = np.sort(xN_Index).reshape(1,xN_Index.shape[0])

total_time = time.time() - beginning

In [14]:
Optimal_value

array([[-116891.62214067]])

In [15]:
total_time

18.73349165916443

In [18]:
beginning2 = time.time()
result = linprog(c, A_eq = A, b_eq = b, method = 'simplex')
total_time2 = time.time() - beginning2

<ipython-input-18-e02b04dcbabb>:2: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  result = linprog(c, A_eq = A, b_eq = b, method = 'simplex')


In [19]:
result.fun

-116891.62214067456

In [9]:
total_time2

0.07688188552856445